In [1]:
#mount the Google Drive to the Colaboratory runtime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# evaluating the performance of machine learning models
from sklearn.metrics import accuracy_score
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt #For creating visualizations in Python
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
#!pip install scikit-learn==1.0.1

In [ ]:
#Reading the file
#data_2009 = pd.read_csv("/content/drive/MyDrive/2009.csv")

In [3]:
#Reading the file for airports and coordinates
data_2015 = pd.read_csv("/content/drive/MyDrive/2015.csv")

In [ ]:
data_2015.info()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data_2015.head(10)

**Taxi Out:** This is the phase where the aircraft moves from the gate or parking area to the runway’s holding point, ready for takeoff. During this phase, pilots start the engines, request permission to taxi, and follow the taxiway markings to reach the designated runway

**Taxi In:** After landing, the aircraft moves from the runway to the gate or parking area where passengers will disembark. This phase is known as “taxi in.” It involves navigating the taxiways and sometimes includes procedures like engine-out taxi-in, which is a fuel-saving measure where one or more engines are shut down during the taxi to save fuel

In [ ]:
print(data_2015.isnull().sum())

In [ ]:
# 'FL_DATE' column to datetime format
data_2015['FL_DATE'] = pd.to_datetime(data_2015['FL_DATE'])

In [ ]:
#'OP_CARRIER' to categorical datatype
#data_2009['OP_CARRIER'] = data_2009['OP_CARRIER'].astype('category')

In [ ]:
#data_2009['ORIGIN'] = data_2009['ORIGIN'].astype(str)
#data_2009['DEST'] = data_2009['DEST'].astype(str)

In [ ]:
total_records = data_2015.shape[0]
total_records

5819079

In [ ]:
print(data_2015['Unnamed: 27'].unique())

[nan]


In [ ]:
cancelled_count = (data_2015['CANCELLED'] == 1).sum()
print(cancelled_count)

89884


In [ ]:
data_2015 = data_2015[data_2015['CANCELLED'] != 1]

In [ ]:
#No data in this attribute
#data_2009.drop(columns=['Unnamed: 27'], inplace=True)

In [ ]:
total_records1 = data_2015.shape[0]
total_records1

5729195

***Replace missing values in 'DEP_DELAY' & 'DEP_TIME' where 'CANCELLED' is 1 with Value as 0, because departure never happened in case of cancelled flights. There are certain records where the Cancelled COde is 1 and still there is departure time and delay mentioned, but all those flights eventually are cancelled because of any reason  $A$, $B$ or $C$***

In [ ]:
#data_2009.loc[data_2009['CANCELLED'] == 1, 'DEP_DELAY'] = 0
#data_2009.loc[data_2009['CANCELLED'] == 1, 'DEP_TIME'] = 0

In [ ]:
#remove the column CANCELLATION_CODE
#data_2009 = data_2009.drop(columns=['CANCELLATION_CODE'])

In [ ]:
#Not required attributes for the study we are conducting:
#data_2009.drop(columns=['DIVERTED'], inplace=True)

In [ ]:
#Not required attributes for the study we are conducting:
#data_2009.drop(columns=['AIR_TIME'], inplace=True)
#data_2009.drop(columns=['DISTANCE'], inplace=True)

In [ ]:
#data_2009.drop(columns=['ACTUAL_ELAPSED_TIME'], inplace=True)

***Imputing values in 'ARR_TIME' and 'ARR_DELAY' with 0 for cancelled flights; as cancelled flights will not arrive at the destination***

In [ ]:
#cancelled_condition = (data_2009['CANCELLED'] == 1) & (data_2009[['ARR_TIME', 'ARR_DELAY']].isnull().any(axis=1))
#data_2009.loc[cancelled_condition, ['ARR_TIME', 'ARR_DELAY']] = 0

In [ ]:
# Now remaining records where flights are not cancelled, but records are having null values in 'ARR_TIME' or 'ARR_DELAY'
#cancelled_unique_counts = data_2009[data_2009[['ARR_TIME', 'ARR_DELAY']].isnull().any(axis=1)]['CANCELLED'].value_counts()
#cancelled_unique_counts

That means all are Not cancelled now. as value is  $0$ .

In [ ]:
# Missing values in 'DEP_DELAY' for rows with missing values in arrival fields
'''
missing_dep_delay = data_2009[data_2009[['ARR_TIME', 'ARR_DELAY']].isnull().any(axis=1)]['DEP_DELAY'].isnull().any()
if missing_dep_delay:
    print("Missing values in 'DEP_DELAY'")
else:
    print("No missing values in 'DEP_DELAY'")'''

In [ ]:
# Missing values in 'CRS_ARR_TIME' for rows with missing values in arrival fields
'''missing_CRS_ARR_TIME = data_2009[data_2009[['ARR_TIME', 'ARR_DELAY']].isnull().any(axis=1)]['DEP_DELAY'].isnull().any()
if missing_CRS_ARR_TIME:
    print("Missing values in 'missing_CRS_ARR_TIME'")
else:
    print("No missing values in 'missing_CRS_ARR_TIME'")'''

***Now just imputing values in ARR_DELAY as computed from ARR_TIME and CRS_ARR_TIME***

In [ ]:
# Filter records where both 'CRS_ARR_TIME' and 'ARR_TIME' are not null
#valid_arrival_records = data_2009[(data_2009['CRS_ARR_TIME'].notnull()) & (data_2009['ARR_TIME'].notnull())]

# Calculate 'ARR_DELAY' using 'CRS_ARR_TIME' and 'ARR_TIME'
#data_2009.loc[valid_arrival_records.index, 'ARR_DELAY'] = valid_arrival_records['ARR_TIME'] - valid_arrival_records['CRS_ARR_TIME']

In [ ]:
#print(data_2009.isnull().sum())

Just to handle one record where ARR_DELAY is 0 and ARR_TIME is missing. We can simply copy the value of CR_ARR_TIME to ARR_TIME.

In [ ]:
# Records where 'ARR_DELAY' is 0 and 'ARR_TIME' is missing
#missing_arr_time_records = data_2009[(data_2009['ARR_DELAY'] == 0) & (data_2009['ARR_TIME'].isnull())]
#missing_arr_time_records

In [ ]:
# Impute missing 'ARR_TIME' with 'CRS_ARR_TIME' for records with 'ARR_DELAY' == 0
#data_2009.loc[missing_arr_time_records.index, 'ARR_TIME'] = missing_arr_time_records['CRS_ARR_TIME']

Now ARR_TIME-2283;ARR_DELAY-2283 are having both same number of missing values.

In [ ]:
#Count number of delays per airline
delays_per_airline = data_2015[data_2015['DEP_DELAY'] > 0].groupby('OP_CARRIER').size().reset_index(name='Number_of_delays')
print(delays_per_airline)

   OP_CARRIER  Number_of_delays
0          AA            245550
1          AS             43541
2          B6            102012
3          DL            282385
4          EV            169503
5          F9             34859
6          HA             20140
7          MQ             93232
8          NK             52033
9          OO            171181
10         UA            256241
11         US             62452
12         VX             23366
13         WN            566583


In [ ]:
print(data_2015.describe())

In [ ]:
#'FL_DATE' column as index: date-based indexing and slicing of the DataFrame.
#data_2015.set_index('FL_DATE', inplace=True)

In [ ]:
# Extract delays and cancellations
#delays = data_2009['DEP_DELAY']
#cancellations = data_2009['CANCELLED']

In [ ]:
# Time series decomposition for delays
decomposition_delays = seasonal_decompose(delays, model='additive', period=365)  # Assuming annual seasonality
trend_delays = decomposition_delays.trend
seasonal_delays = decomposition_delays.seasonal
residual_delays = decomposition_delays.resid

In [ ]:
# Plot the decomposition of delays
plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(delays, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend_delays, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal_delays, label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual_delays, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

# Time series decomposition for cancellations
decomposition_cancellations = seasonal_decompose(cancellations, model='additive', period=365)  # Assuming annual seasonality
trend_cancellations = decomposition_cancellations.trend
seasonal_cancellations = decomposition_cancellations.seasonal
residual_cancellations = decomposition_cancellations.resid

# Plot the decomposition of cancellations
plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(cancellations, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend_cancellations, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal_cancellations, label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual_cancellations, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

# Apply ARIMA model for forecasting delays
model_delays = ARIMA(delays, order=(5,1,0))  # Example order, you may need to tune this
results_delays = model_delays.fit()
forecast_delays = results_delays.predict(start=len(delays), end=len(delays) + 30, typ='levels')  # Example forecast for the next 30 days

# Plot the forecasted delays
plt.figure(figsize=(12, 6))
plt.plot(delays, label='Original')
plt.plot(forecast_delays, label='Forecast')
plt.legend(loc='best')
plt.title('Forecasted Delays')
plt.xlabel('Date')
plt.ylabel('Delay (minutes)')
plt.show()

In [8]:
# New column 'DELAYED' to indicate whether the flight is delayed or not
data_2015['DELAYED'] = data_2015['DEP_DELAY'] > 0
print(data_2015[['DEP_DELAY', 'DELAYED']].head())

   DEP_DELAY  DELAYED
0       -4.0    False
1       14.0     True
2       12.0     True
3       11.0     True
4       -3.0    False


In [9]:
delayed_flights = data_2015[data_2015['DELAYED'] == True].shape[0]
print(delayed_flights)
Ontime_flights = data_2015[data_2015['DELAYED'] == False].shape[0]
print(Ontime_flights)

2125618
3693461


In [ ]:
data_2015.columns

In [10]:
# Specify the columns to include in the separate dataset
columns_to_include = ['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'WEATHER_DELAY', 'DELAYED']

# Create a separate dataset with the specified columns
airline_delay = data_2015.loc[:, columns_to_include]

In [ ]:
airline_delay.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,WEATHER_DELAY,DELAYED
0,2015-01-01,NK,195,MCO,FLL,2147,2143.0,-4.0,NaN,False
1,2015-01-01,NK,197,LGA,FLL,1050,1104.0,14.0,NaN,True
2,2015-01-01,NK,198,FLL,MCO,700,712.0,12.0,0.0,True
3,2015-01-01,NK,199,IAH,LAS,2240,2251.0,11.0,NaN,True
4,2015-01-01,NK,200,IAH,ORD,623,620.0,-3.0,NaN,False


In [ ]:
airline_delay['FL_DATE'] = pd.to_datetime(airline_delay['FL_DATE'])
airline_delay['FL_DATE_month'] = airline_delay['FL_DATE'].dt.month
airline_delay['FL_DATE_weekday'] = airline_delay['FL_DATE'].dt.day_name()
airline_delay['FL_DATE_weekend'] = airline_delay['FL_DATE'].dt.dayofweek // 5

In [11]:
!pip install meteostat

In [14]:
from datetime import datetime
from meteostat import Point
from meteostat import Hourly
from meteostat import Daily
from meteostat import Monthly

from prophet import Prophet
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_absolute_error

In [ ]:
# Set time period
start = datetime(2009, 1, 1)
end = datetime(2009, 3, 2)
# Create Point for DCA
DCA = Point(38.85208, -77.03772)
data = Daily(DCA, start, end)
data = data.fetch()

In [ ]:
print(data)

In [12]:
#Reading the file for airports and coordinates
Airport_Locations = pd.read_csv("/content/drive/MyDrive/Airport_Locations.csv")

In [15]:
def fetch_daily_weather_data(Airport_Locations, start_date, end_date):
    """
    Fetch daily weather data for each airport listed in the Airport_Locations dataset.

    Parameters:
        airport_locations_df (DataFrame): DataFrame containing airport codes, latitude, and longitude.
        start_date (datetime): Start date for the weather data retrieval.
        end_date (datetime): End date for the weather data retrieval.

    Returns:
        DataFrame: DataFrame containing daily weather data for each airport.
    """
    # Initialize an empty list to store weather data for each airport
    airport_weather_data = []

    # Iterate over each row of the Airport_Locations DataFrame
    for index, row in Airport_Locations.iterrows():
        # Extract airport code, latitude, and longitude from the current row
        airport_code = row['Airport Code']
        latitude = row['Latitude']
        longitude = row['Longitude']

        # Create Point object for the airport location
        airport_point = Point(latitude, longitude)

        # Fetch daily weather data for the airport location
        airport_weather = Daily(airport_point, start=start_date, end=end_date)
        airport_weather = airport_weather.fetch()

        # Add airport code to each daily weather data record
        airport_weather['Airport Code'] = airport_code

        # Append the airport weather data to the list
        airport_weather_data.append(airport_weather)

    # Concatenate the list of DataFrames into a single DataFrame
    airport_weather_df = pd.concat(airport_weather_data)

    return airport_weather_df

# Example usage:
# Set time period
start_date = datetime(2015, 1, 1)
end_date = datetime(2015, 12, 31)

# Fetch daily weather data for each airport in Airport_Locations dataset
airport_weather_dataset = fetch_daily_weather_data(Airport_Locations, start_date, end_date)

# Display the resulting dataset
print(airport_weather_dataset)

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
2015-01-01   1.7  -2.7   8.3   0.0   0.0  210.0  13.3   NaN  1021.6   NaN   
2015-01-02   5.3   1.7   9.4   0.0   0.0    NaN   4.7   NaN  1025.5   NaN   
2015-01-03   4.0   1.1   5.6  12.7   0.0    NaN   5.4   NaN     NaN   NaN   
2015-01-04   9.8   5.6  19.4   5.1   0.0  209.0  20.2   NaN  1012.5   NaN   
2015-01-05   7.8  -1.6  11.1   0.0   0.0  310.0  23.0   NaN  1025.3   NaN   
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...   
2015-12-26   5.5  -1.1   8.3   NaN   NaN   27.0  15.6   NaN  1016.7   NaN   
2015-12-27  -2.5  -3.3  -1.1   0.0   NaN   22.0  27.0   NaN  1027.8   NaN   
2015-12-28  -4.8  -6.1  -2.8   NaN   NaN    NaN  18.5   NaN     NaN   NaN   
2015-12-29  -4.8 -10.6  -1.1   0.0   NaN    NaN   3.7   NaN  1020.5   NaN   
2015-12-30  -2.2  -5.0   0.0   0.0   NaN    NaN   2.6   NaN  1026.1   NaN   

           Airport Code  
2015-01-01          DCA  
2015-01-02          DCA

In [16]:
airport_weather_dataset.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Airport Code
2015-01-01,1.7,-2.7,8.3,0.0,0.0,210.0,13.3,NaN,1021.6,NaN,DCA
2015-01-02,5.3,1.7,9.4,0.0,0.0,NaN,4.7,NaN,1025.5,NaN,DCA
2015-01-03,4.0,1.1,5.6,12.7,0.0,NaN,5.4,NaN,NaN,NaN,DCA
2015-01-04,9.8,5.6,19.4,5.1,0.0,209.0,20.2,NaN,1012.5,NaN,DCA
2015-01-05,7.8,-1.6,11.1,0.0,0.0,310.0,23.0,NaN,1025.3,NaN,DCA


In [20]:
#file_path = 'path/to/your/file.csv'  # Replace 'path/to/your/file.csv' with the desired file path and name
import os
os.chdir('/content/drive/MyDrive')
# Write the DataFrame to a CSV file
#df.to_csv(file_path, index=False)
airport_weather_dataset.to_csv('airport_weather_dataset.csv', index=False)

In [ ]:
airport_weather_dataset.columns

Index(['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun', 'Airport Code'],
      dtype='object')

In [ ]:
# Reset the index to convert the date index into a regular column
airport_weather_dataset.reset_index(inplace=True)

# Display the updated airport_weather_dataset DataFrame
print(airport_weather_dataset)

In [ ]:
airport_weather_dataset.rename(columns={'index': 'Date'}, inplace=True)

In [ ]:
airport_weather_dataset.head()

,Date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Airport Code
0,2015-01-01,1.7,-2.7,8.3,0.0,0.0,210.0,13.3,NaN,1021.6,NaN,DCA
1,2015-01-02,5.3,1.7,9.4,0.0,0.0,NaN,4.7,NaN,1025.5,NaN,DCA
2,2015-01-03,4.0,1.1,5.6,12.7,0.0,NaN,5.4,NaN,NaN,NaN,DCA
3,2015-01-04,9.8,5.6,19.4,5.1,0.0,209.0,20.2,NaN,1012.5,NaN,DCA
4,2015-01-05,7.8,-1.6,11.1,0.0,0.0,310.0,23.0,NaN,1025.3,NaN,DCA


In [ ]:
# Merge tavg, tmin, and tmax from airport_weather_dataset into airline_delay DataFrame
airline_delay = pd.merge(airline_delay, airport_weather_dataset[['Date', 'Airport Code', 'tavg', 'prcp', 'snow','wspd']],
                         left_on=['FL_DATE', 'ORIGIN'], right_on=['Date', 'Airport Code'],
                         how='left', suffixes=('_airline', '_weather'))

# Display the updated airline_delay DataFrame
print(airline_delay)

In [ ]:
airline_delay.drop(columns=['Date', 'Airport Code'], inplace=True)

In [ ]:
airline_delay.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,WEATHER_DELAY,DELAYED,FL_DATE_month,FL_DATE_weekday,FL_DATE_weekend,tavg,prcp,snow,wspd
0,2015-01-01,NK,195,MCO,FLL,2147,2143.0,-4.0,NaN,False,1,Thursday,0,18.3,0.0,0.0,12.6
1,2015-01-01,NK,197,LGA,FLL,1050,1104.0,14.0,NaN,True,1,Thursday,0,-1.7,0.0,0.0,20.2
2,2015-01-01,NK,198,FLL,MCO,700,712.0,12.0,0.0,True,1,Thursday,0,22.8,4.6,NaN,8.6
3,2015-01-01,NK,199,IAH,LAS,2240,2251.0,11.0,NaN,True,1,Thursday,0,6.3,3.8,0.0,14.0
4,2015-01-01,NK,200,IAH,ORD,623,620.0,-3.0,NaN,False,1,Thursday,0,6.3,3.8,0.0,14.0


In [ ]:
print(airline_delay.isnull().sum())

FL_DATE                    0
OP_CARRIER                 0
OP_CARRIER_FL_NUM          0
ORIGIN                     0
DEST                       0
CRS_DEP_TIME               0
DEP_TIME                   0
DEP_DELAY                  0
WEATHER_DELAY        4665756
DELAYED                    0
FL_DATE_month              0
FL_DATE_weekday            0
FL_DATE_weekend            0
tavg                   79960
prcp                  163331
snow                 1293540
wspd                   82058
dtype: int64


In [ ]:
#Reading the file for airports and coordinates
Holidays_2015 = pd.read_csv("/content/drive/MyDrive/Holidays_2015.csv")

In [ ]:
Holidays_2015.head()

,Date
0,2015-01-01
1,2015-01-19
2,2015-02-16
3,2015-05-10
4,2015-05-25


In [18]:
os.chdir('/content/drive/MyDrive')

NameError: name 'os' is not defined

In [ ]:
Holidays_2015['Date'] = pd.to_datetime(Holidays_2015['Date'])

In [ ]:
# Create a new column 'Statutory_Holiday' in airline_delay DataFrame and initialize it with 0
airline_delay['Statutory_Holiday'] = 0

In [ ]:
# Merge Holidays_2015 with airline_delay based on the 'FL_DATE' column
airline_delay = pd.merge(airline_delay, Holidays_2015, left_on='FL_DATE', right_on='Date', how='left')

# Fill NaN values in the merged column with 0 (indicating non-holiday)
airline_delay['Statutory_Holiday'] = airline_delay['Statutory_Holiday'].fillna(0)

# Convert 'Statutory_Holiday' column to integer type
airline_delay['Statutory_Holiday'] = airline_delay['Statutory_Holiday'].astype(int)

In [ ]:
airline_delay.drop(columns=['Date_y'], inplace=True)

In [ ]:
airline_delay.head(10)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,WEATHER_DELAY,DELAYED,FL_DATE_month,FL_DATE_weekday,FL_DATE_weekend,tavg,prcp,snow,wspd,Date_x,Statutory_Holiday
0,2015-01-01,NK,195,MCO,FLL,2147,2143.0,-4.0,NaN,False,1,Thursday,0,18.3,0.0,0.0,12.6,2015-01-01,0
1,2015-01-01,NK,197,LGA,FLL,1050,1104.0,14.0,NaN,True,1,Thursday,0,-1.7,0.0,0.0,20.2,2015-01-01,0
2,2015-01-01,NK,198,FLL,MCO,700,712.0,12.0,0.0,True,1,Thursday,0,22.8,4.6,NaN,8.6,2015-01-01,0
3,2015-01-01,NK,199,IAH,LAS,2240,2251.0,11.0,NaN,True,1,Thursday,0,6.3,3.8,0.0,14.0,2015-01-01,0
4,2015-01-01,NK,200,IAH,ORD,623,620.0,-3.0,NaN,False,1,Thursday,0,6.3,3.8,0.0,14.0,2015-01-01,0
5,2015-01-01,NK,201,FLL,STT,1253,1258.0,5.0,NaN,True,1,Thursday,0,22.8,4.6,NaN,8.6,2015-01-01,0
6,2015-01-01,NK,202,DFW,BWI,1342,1337.0,-5.0,NaN,False,1,Thursday,0,0.7,15.0,0.0,10.1,2015-01-01,0
7,2015-01-01,NK,203,BOS,PBI,1555,1554.0,-1.0,NaN,False,1,Thursday,0,-3.3,0.0,0.0,23.0,2015-01-01,0
8,2015-01-01,NK,204,PBI,BOS,1205,1204.0,-1.0,NaN,False,1,Thursday,0,23.4,5.8,NaN,8.3,2015-01-01,0
9,2015-01-01,NK,209,ORD,OAK,1534,1534.0,0.0,NaN,False,1,Thursday,0,-6.7,0.0,0.0,23.4,2015-01-01,0


In [ ]:
# Export DataFrame to CSV
airline_delay.to_csv('airline_delay.csv', index=False)

In [ ]:
import os

# Get the current working directory
cwd = os.getcwd()
print("Current working directory:", cwd)

Current working directory: /content
